In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
np.set_printoptions(precision=3, formatter={'float': '{:0.2e}'.format})

## Goal
Plug in 2 beam powers, 2 waists, 1 detuning, species, transport time. Outputs single and full-beam trap depth, fringe contrast, both trap frequencies, scattering rate, and heating due to scattering. Also outputs polarizabilities of ground and excited states at given detuning.
## Imports & functions

In [3]:
datas = pd.read_csv('Cs1Pol/Cs1_6s.csv', header = 1, delimiter = ',')
datap1 = pd.read_csv('Cs1Pol/Cs1_6p1.csv', header = 1, delimiter = ',')
datap3 = pd.read_csv('Cs1Pol/Cs1_6p3.csv', header = 1, delimiter = ',')

In [4]:
d2freq = 351.726
d1freq = 335.116
c = 299792458

gamma = 4.57 # MHz (everything in cyclic frequencies)
Isat_D1 = 25 # W/m^2
Isat_D2 = 17
kB = 1.381*10**(-23)
frec = .0019 # MHz
recoil_speed = .0034 # m/s
m = 2.2069*1e-25 # kg
recoil_temp = .18 # uK
wave = 894 # nm

In [5]:
def wave_to_freq(wave):
    return (c/wave/1000)

def calc_scatt_rate(det, I):
    return np.pi*gamma*(I)/(1+I+4*det**2/gamma**2)

def calc_trap_depth(polarizability, I, Isat):
    return polarizability*I*Isat/2/0.00265441873/kB/(6.06510057*10**(40))

def calc_lifetime(delta_d1):
    delta_d1 = delta_d1*1e6
    delta_d2 = delta_d1 + (d1freq - d2freq)*1e6

    tau_d1 = np.abs(delta_d1 / 4 / np.pi / frec / gamma)
    tau_d2 = np.abs(delta_d2 / 4 / np.pi / frec / gamma)
    tau_tot = 1 / (1 / tau_d1 + 1 / tau_d2)
    return tau_tot/1e3

In [6]:
def calc_all_params(inputs, print_all = False):
    [Podt, Plat, wodt, wlat, delta, t] = inputs
    
    # Polarizability at some detuning
    idxs = (wave_to_freq(datas['Wavelength'])*1e3-d1freq*1e3-delta).abs().idxmin()
    pol_s = datas.loc[idxs]['alpha_0']
    
    idxp1 = (wave_to_freq(datap1['Wavelength'])*1e3-d1freq*1e3-delta).abs().idxmin()
    pol_p1 = datap1.loc[idxp1]['alpha_0']
    
    idxp3 = (wave_to_freq(datap3['Wavelength'])*1e3-d1freq*1e3-delta).abs().idxmin()
    pol_p3s = datap3.loc[idxp3]['alpha_0']
    pol_p3t = datap3.loc[idxp3]['alpha_2']
    
    pols = [pol_s, pol_p1, pol_p3s, pol_p3t]
    
    # Trap depths & contrast
    Iodt = 2*Podt/1e3/np.pi/(wodt/1e6)**2
    Ilat = 2*Plat/1e3/np.pi/(wlat/1e6)**2
    Itot = (np.sqrt(Iodt)+np.sqrt(Ilat))**2
    Imin = (np.sqrt(Iodt)-np.sqrt(Ilat))**2
    
    single_trap_depth = calc_trap_depth(pol_s, Iodt, 1)*1e6 # Iodt already includes dimensions, so no need to add in Isat
    full_trap_depth = calc_trap_depth(pol_s, Itot, 1)*1e6
    min_trap_depth = calc_trap_depth(pol_s, Imin, 1)*1e6
    fringe_contrast = full_trap_depth - min_trap_depth
    
    depths = [single_trap_depth, full_trap_depth, fringe_contrast]
    
    # Scattering rate
    single_beam_scatt = calc_scatt_rate(delta*1e3, Iodt/Isat_D1)*1e3 + calc_scatt_rate(delta*1e3 + (d1freq - d2freq)*1e6, Iodt/Isat_D2)*1e3
    full_scatt = calc_scatt_rate(delta*1e3, Itot/Isat_D1)*1e3 + calc_scatt_rate(delta*1e3 + (d1freq - d2freq)*1e6, Itot/Isat_D2)*1e3
    
    # Lifetime from scattering
    lifetime_scatt = calc_lifetime(delta/1e3)
    
    # Push and heating
    push = single_beam_scatt * t * recoil_speed
    diff = full_scatt * recoil_temp * 2
    
    scatt = [single_beam_scatt, full_scatt, lifetime_scatt, push, diff]
    
    # Radial trap frequency
    f_rad = np.sqrt(4*kB*single_trap_depth/1e6/m/(wodt/1e6)**2)/2/np.pi/1e3
    
    # Axial trap frequency
    f_ax = 1 / (wave/1e9) * np.sqrt(2*fringe_contrast/1e6*kB/m)/1e3
    
    freqs = [f_rad, f_ax]
    
    if print_all:
        print('Polarizabilities (au)')
        print('    S: ', format(pol_s, ".1e"))
        print('    P1/2: ', format(pol_p1, ".1e"))
        print('    P3/2: ', format(pol_p3s, ".1e"))
        print('    P3/2, tensor: ', format(pol_p3t, ".1e"))
        print(' ')
        print('Trap Depths (uK)')
        print('    Single beam: ', round(single_trap_depth, 0))
        print('    Full trap: ', round(full_trap_depth, 0))
        print('    Fringe contrast: ', round(fringe_contrast, 0))
        print(' ')
        print('Scattering Rates (kHz)')
        print('    Single beam: ', round(single_beam_scatt, 1))
        print('    Full trap: ', round(full_scatt, 1))
        print(' ')
        print('Lifetime from scattering: ', round(lifetime_scatt, 1), ' ms')
        print('Scattering push (single beam only): ', round(push, 1), ' m/s')
        print('Scattering heating rate: ', round(diff, 1), ' uK/ms')
        print(' ')
        print('Trap Frequencies (kHz)')
        print('    Radial: ', round(f_rad, 2))
        print('    Axial: ', round(f_ax, 2))
        
    return pols, depths, scatt, freqs

In [7]:
res = calc_all_params([100, 100, 100, 400, -100, 25], print_all = True)

Polarizabilities (au)
    S:  2.2e+05
    P1/2:  -2.2e+05
    P3/2:  -4.1e+03
    P3/2, tensor:  1.7e+03
 
Trap Depths (uK)
    Single beam:  322.0
    Full trap:  502.0
    Fringe contrast:  322.0
 
Scattering Rates (kHz)
    Single beam:  1.9
    Full trap:  3.0
 
Lifetime from scattering:  911.0  ms
Scattering push (single beam only):  0.2  m/s
Scattering heating rate:  1.1  uK/ms
 
Trap Frequencies (kHz)
    Radial:  0.45
    Axial:  224.39


### Heating from Phase Noise

In [ ]:
def noise_heating_lifetime(f_ax, S, L): # returns uK/ms, input axial trap freq in kHz
    rate = np.pi / 2 * m * (2 * np.pi * f_ax*1e3)**4 / kB * S * (wave/1e9 / (4 * np.pi))**2 * (L / c)**2
    return rate*1e3

def white_noise(nu):
    return nu/np.pi

In [ ]:
# Realistically, we currently have 1 m propagation difference, worst case 10 MHz linewidth, 800 kHz trap frequency
print(str(round(noise_heating_lifetime(300, white_noise(10e6), .8), 0)), ' uK/ms')
# Achievable with this laser and some care
print(str(round(noise_heating_lifetime(300, white_noise(5e6), .4), 1)), ' uK/ms')
# With a decent ECDL, 500 kHz linewidth
print(str(round(noise_heating_lifetime(300, white_noise(.5e6), .2), 2)), ' uK/ms')
# Better laser but without decreased path difference
print(str(round(noise_heating_lifetime(300, white_noise(.5e6), 1), 2)), ' uK/ms')

### Where I've been working
Retro with 1% up to 65% efficiency -> axial frequencies from 500 kHz to 1.8 MHz, with ~2 m path length difference. Heating between 13 uK/ms at minimum and 2 mK/ms for a 1 MHz linewidth laser. Even at 10 mW retro, 310 uK/ms. This would absolutely explain my measured lattice lifetimes.
### Where do I want to work?
#### Achievable Parameter Sets

In [ ]:
# Where we started
det = -50 # GHz
transport_time = 25 # ms
Podt = 45 # mW
Plat = 65 # mW
wodt = 100 # um
wlat = 400 # um

pols, depths, scatt, freqs = calc_all_params([Podt, Plat, wodt, wlat, det, transport_time], print_all=True)

In [ ]:
# Scanning lattice power
Plat_list = np.logspace(-1, 2, 100)

pols, depths, scatt, freqs = calc_all_params([Podt, Plat_list, wodt, wlat, det, transport_time])
f_rad, f_ax = freqs
d1, d2, contrast = depths
#print('Single beam trap depth: ' + str(np.round(d1, 0)) + ' uK')

fig, ax = plt.subplots(2, sharex = True)
ax[0].plot(Plat_list, f_ax)
ax[1].plot(Plat_list, contrast)

ax[1].set_xlabel('Lattice beam power (mW)')
ax[0].set_ylabel('Axial trap frequency (kHz)')
ax[1].set_ylabel('Fringe contrast ($\mu$K)')

#### Heating Rates

In [ ]:
fig, ax = plt.subplots()
noise_list = np.logspace(3, 7, 5)
freqs = np.linspace(1, 4e2, 100)

ax.hlines(xmin=0, xmax=400, y=1, color = 'black', linestyle = 'dotted')

for nu in noise_list:
    ax.plot(freqs, noise_heating_lifetime(freqs, white_noise(nu), 2), label = str(round(nu/1e6, 3)) + ' MHz')

ax.set_yscale('log')
ax.set_ylim(1e-6, 1e3)
ax.set_xlabel('Axial Trap Frequency (kHz)')
ax.set_ylabel('Heating Rate ($\mu$K/ms)')
ax.legend(title = 'Linewidth')

In [ ]:
fig, ax = plt.subplots()
nu = 5e6
print('With a ', round(nu/1e6, 0), ' MHz linewidth laser')
L_list = np.logspace(-2, 1, 4)
freqs = np.linspace(1, 4e2, 100)

ax.hlines(xmin=0, xmax=400, y=1, color = 'black', linestyle = 'dotted')

for L in L_list:
    ax.plot(freqs, noise_heating_lifetime(freqs, white_noise(nu), L), label = str(round(L, 2)) + ' m')

ax.set_yscale('log')
ax.set_ylim(1e-6, 1e3)
ax.set_xlabel('Axial Trap Frequency (kHz)')
ax.set_ylabel('Heating Rate ($\mu$K/ms)')
ax.legend(title = 'Path Length Difference')

In [ ]:
det = -50 # GHz
transport_time = 25 # ms
Podt = 45 # mW
Plat = 65 # mW
wodt = 100 # um
wlat = 400 # um

pols, depths, scatt, freqs = calc_all_params([Podt, Plat, wodt, wlat, det, transport_time], print_all=True)
print()
print(str(round(noise_heating_lifetime(freqs[1], white_noise(.5e6), .2), 2)), ' uK/ms')

In [ ]:
dets = [-100, -150, -200, -250, -300]
pow_arr = np.linspace(0, 300, 100)
fig, ax = plt.subplots(3, 2, figsize = (8, 12), sharex = True)

for det in dets:
    Podt = pow_arr
    Plat = pow_arr
    pols, depths, scatt, freqs = calc_all_params([Podt, Plat, wodt, wlat, det, transport_time], print_all=False)
    ax[0, 0].plot(pow_arr, depths[1])
    ax[0, 1].plot(pow_arr, depths[2], label = str(-det) + ' GHz')
    ax[1, 0].plot(pow_arr, scatt[0])
    ax[1, 1].plot(pow_arr, scatt[3])
    ax[2, 0].plot(pow_arr, freqs[0])
    ax[2, 1].plot(pow_arr, freqs[1])

ax[0, 0].set_ylabel('Trap Depth ($\mu$K)')
ax[0, 1].set_ylabel('Fringe Contrast ($\mu$K)')
ax[1, 0].set_ylabel('Scattering Rate (kHz)')
ax[1, 1].set_ylabel('Push from Scattering (m/s)')
ax[2, 0].set_ylabel('Radial Trap Frequency (kHz)')
ax[2, 1].set_ylabel('Axial Trap Frequency (kHz)')

ax[0, 0].set_ylim([0, 800])
ax[0, 1].set_ylim([0, 800])
ax[1, 0].set_ylim([0, 3])
ax[1, 1].set_ylim([0, .5])
ax[2, 0].set_ylim([0, 1])
ax[2, 1].set_ylim([0, 350])

ax[0, 1].legend(title = 'Detuning')
ax[0, 0].set_xlim([0, 300])
ax[2, 0].set_xlabel('Power (mW)')
ax[2, 1].set_xlabel('Power (mW)')
fig.tight_layout()

In [ ]:
def calc_total_lifetime(heating_rates, init_temp, trap_depth):
    total_heating = np.sum(heating_rates)
    return (trap_depth - init_temp) / total_heating

In [ ]:
print(calc_total_lifetime([2.6, .16], 25, 500))